In [122]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import random

In [123]:
pd.__version__

'1.4.4'

Se a gente lê o arquivo de 3GB inteiro, vai dar problema. Aqui tem uma receita para ler somente x%: https://www.kaggle.com/questions-and-answers/53925 

In [124]:
def skip(i,fraction=0.01):
    if i == 0:
        return False  # precisamos manter o row=0 para ter o header
    else:
        return random.random() > fraction

[skip(i) for i in range(10)]

[False, True, True, True, True, True, True, True, True, True]

In [125]:
%time df = pd.read_csv('../inep/enem/2019/DADOS/MICRODADOS_ENEM_2019.csv',skiprows = skip, encoding='latin1',sep=";")

CPU times: user 24.5 s, sys: 540 ms, total: 25 s
Wall time: 25 s


In [126]:
df

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004658,2019,1502939,Dom Eliseu,15,PA,17,F,1,3,...,A,A,A,B,B,B,C,A,A,A
1,190001004685,2019,1503606,Itaituba,15,PA,25,M,0,0,...,B,A,B,C,A,A,B,A,B,B
2,190001004894,2019,1501402,Belém,15,PA,23,F,1,2,...,A,A,A,B,A,A,A,A,A,A
3,190001004925,2019,1506302,Salvaterra,15,PA,18,F,1,2,...,A,A,A,B,A,A,C,A,A,A
4,190001004967,2019,1506807,Santarém,15,PA,20,F,1,3,...,A,A,A,B,A,A,D,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50673,190006119978,2019,2930105,Senhor do Bonfim,29,BA,16,F,1,2,...,B,A,A,B,A,A,D,A,A,B
50674,190006120038,2019,2912103,Ibicaraí,29,BA,17,F,3,4,...,A,A,A,A,A,A,B,A,A,B
50675,190006120062,2019,2927408,Salvador,29,BA,18,F,1,3,...,B,A,A,B,A,A,D,A,A,B
50676,190006120091,2019,3155603,Rio Pardo de Minas,31,MG,17,M,1,3,...,A,A,A,B,B,A,C,A,B,B


In [127]:
df.describe()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,NU_IDADE,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,...,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q005
count,5.067800e+04,50678.0,5.067800e+04,50678.000000,50678.000000,50678.000000,50678.000000,50678.000000,4.922300e+04,49223.000000,...,36927.000000,50678.000000,39023.000000,39023.000000,39023.000000,39023.000000,39023.000000,39023.000000,39023.000000,50678.000000
mean,1.900036e+11,2019.0,3.085661e+06,30.705691,22.096314,1.076463,2.133273,1.030447,3.059089e+06,30.439266,...,523.536656,0.521015,1.140430,120.503293,122.662532,114.664172,121.656972,91.446070,570.933039,3.875192
std,1.467341e+06,0.0,9.959533e+05,9.928975,7.476409,0.442095,1.018001,0.208546,9.832133e+05,9.798202,...,109.223437,0.499563,0.793735,33.473397,43.019092,40.207477,38.213816,59.601615,189.179238,1.500371
min,1.900010e+11,2019.0,1.100015e+06,11.000000,13.000000,0.000000,0.000000,0.000000,1.100015e+06,11.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.900023e+11,2019.0,2.404705e+06,24.000000,18.000000,1.000000,1.000000,1.000000,2.314102e+06,23.000000,...,435.000000,0.000000,1.000000,120.000000,120.000000,100.000000,100.000000,40.000000,480.000000,3.000000
50%,1.900036e+11,2019.0,3.125101e+06,31.000000,19.000000,1.000000,2.000000,1.000000,3.112307e+06,31.000000,...,501.000000,1.000000,1.000000,120.000000,120.000000,120.000000,120.000000,100.000000,580.000000,4.000000
75%,1.900048e+11,2019.0,3.549904e+06,35.000000,24.000000,1.000000,3.000000,1.000000,3.548807e+06,35.000000,...,599.600000,1.000000,1.000000,140.000000,140.000000,140.000000,140.000000,140.000000,680.000000,5.000000
max,1.900061e+11,2019.0,5.300108e+06,53.000000,80.000000,4.000000,5.000000,4.000000,5.300108e+06,53.000000,...,985.000000,1.000000,9.000000,200.000000,200.000000,200.000000,200.000000,200.000000,1000.000000,20.000000


In [128]:
df.sample(5)

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
34033,190004430248,2019,2300408,Aiuaba,23,CE,17,M,1,3,...,A,A,A,B,B,A,B,A,A,A
39074,190004939821,2019,2112209,Timon,21,MA,18,M,1,3,...,A,A,A,B,A,A,D,A,A,B
14071,190002440278,2019,3304557,Rio de Janeiro,33,RJ,19,M,1,3,...,A,A,A,B,A,A,C,A,B,B
39775,190005006904,2019,3304557,Rio de Janeiro,33,RJ,23,M,1,1,...,B,A,B,C,A,A,E,A,C,B
46575,190005692381,2019,2205003,Itainópolis,22,PI,20,F,1,2,...,A,A,A,B,B,A,D,A,A,A


In [129]:
df["TP_PRESENCA_CN"].value_counts()

1    36927
0    13737
2       14
Name: TP_PRESENCA_CN, dtype: int64

In [130]:
def filterdf(df):
    df = df[df["TP_PRESENCA_CN"] == 1] #presente nas 4 provas
    df = df[df["TP_PRESENCA_CH"] == 1]
    df = df[df["TP_PRESENCA_LC"] == 1]
    df = df[df["TP_PRESENCA_MT"] == 1]
    df = df[df["IN_TREINEIRO"] == 0] # não é treineiro
    df = df[df["TP_ST_CONCLUSAO"].isin([1,2])] #afirma que concluiu ou vai concluir EM em 2019
    return df

dff = filterdf(df)
dff

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
4,190001004967,2019,1506807,Santarém,15,PA,20,F,1,3,...,A,A,A,B,A,A,D,A,A,A
5,190001005059,2019,1506807,Santarém,15,PA,17,F,1,4,...,A,A,A,B,A,A,B,A,A,A
6,190001005138,2019,1508100,Tucuruí,15,PA,19,M,1,3,...,A,A,A,B,A,A,C,A,B,B
7,190001005147,2019,1505205,Oeiras do Pará,15,PA,37,F,1,3,...,A,A,A,B,A,A,C,A,A,A
8,190001005219,2019,1502954,Eldorado do Carajás,15,PA,18,F,1,3,...,A,A,A,B,A,A,C,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50671,190006119878,2019,2610608,Paudalho,26,PE,19,F,1,0,...,B,A,A,B,A,A,D,A,A,A
50672,190006119928,2019,2700300,Arapiraca,27,AL,26,F,2,3,...,B,A,B,B,A,A,C,A,B,B
50674,190006120038,2019,2912103,Ibicaraí,29,BA,17,F,3,4,...,A,A,A,A,A,A,B,A,A,B
50675,190006120062,2019,2927408,Salvador,29,BA,18,F,1,3,...,B,A,A,B,A,A,D,A,A,B


In [131]:
dff

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
4,190001004967,2019,1506807,Santarém,15,PA,20,F,1,3,...,A,A,A,B,A,A,D,A,A,A
5,190001005059,2019,1506807,Santarém,15,PA,17,F,1,4,...,A,A,A,B,A,A,B,A,A,A
6,190001005138,2019,1508100,Tucuruí,15,PA,19,M,1,3,...,A,A,A,B,A,A,C,A,B,B
7,190001005147,2019,1505205,Oeiras do Pará,15,PA,37,F,1,3,...,A,A,A,B,A,A,C,A,A,A
8,190001005219,2019,1502954,Eldorado do Carajás,15,PA,18,F,1,3,...,A,A,A,B,A,A,C,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50671,190006119878,2019,2610608,Paudalho,26,PE,19,F,1,0,...,B,A,A,B,A,A,D,A,A,A
50672,190006119928,2019,2700300,Arapiraca,27,AL,26,F,2,3,...,B,A,B,B,A,A,C,A,B,B
50674,190006120038,2019,2912103,Ibicaraí,29,BA,17,F,3,4,...,A,A,A,A,A,A,B,A,A,B
50675,190006120062,2019,2927408,Salvador,29,BA,18,F,1,3,...,B,A,A,B,A,A,D,A,A,B


In [132]:
# salvar a amostra filtrada em disco
dff.to_csv('data/enem19-1perc2.csv',index=False)

In [133]:
# Selecionar apenas as colunas que desejamos trabalahr
df_selecao = dff[["NU_INSCRICAO","NU_ANO","CO_MUNICIPIO_RESIDENCIA","NO_MUNICIPIO_RESIDENCIA","CO_UF_RESIDENCIA","SG_UF_RESIDENCIA","TP_COR_RACA","TP_SEXO","NU_IDADE","TP_ENSINO","TP_ESCOLA","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]]
df_selecao

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_ENSINO,TP_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_MT,NU_NOTA_LC,NU_NOTA_REDACAO
4,190001004967,2019,1506807,Santarém,15,PA,3,F,20,1.0,1,537.9,582.2,417.7,585.7,780.0
5,190001005059,2019,1506807,Santarém,15,PA,4,F,17,1.0,2,416.2,569.5,459.9,539.6,660.0
6,190001005138,2019,1508100,Tucuruí,15,PA,3,M,19,NaN,1,526.9,589.7,708.5,588.7,860.0
7,190001005147,2019,1505205,Oeiras do Pará,15,PA,3,F,37,1.0,1,376.8,502.1,392.1,546.5,620.0
8,190001005219,2019,1502954,Eldorado do Carajás,15,PA,3,F,18,1.0,1,377.6,443.8,425.0,433.5,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50671,190006119878,2019,2610608,Paudalho,26,PE,0,F,19,1.0,1,419.4,354.0,587.7,461.2,520.0
50672,190006119928,2019,2700300,Arapiraca,27,AL,3,F,26,1.0,1,567.1,513.7,551.0,559.2,460.0
50674,190006120038,2019,2912103,Ibicaraí,29,BA,4,F,17,NaN,2,340.9,497.7,372.7,386.8,260.0
50675,190006120062,2019,2927408,Salvador,29,BA,3,F,18,1.0,2,397.6,438.5,450.2,476.9,300.0


## Eliminar as colunas que possuem nota 0

In [134]:
#Salvar colunas selecionadas
df_selecao.to_csv('data/selecao.csv',index=False)
df_selecao

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_ENSINO,TP_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_MT,NU_NOTA_LC,NU_NOTA_REDACAO
4,190001004967,2019,1506807,Santarém,15,PA,3,F,20,1.0,1,537.9,582.2,417.7,585.7,780.0
5,190001005059,2019,1506807,Santarém,15,PA,4,F,17,1.0,2,416.2,569.5,459.9,539.6,660.0
6,190001005138,2019,1508100,Tucuruí,15,PA,3,M,19,NaN,1,526.9,589.7,708.5,588.7,860.0
7,190001005147,2019,1505205,Oeiras do Pará,15,PA,3,F,37,1.0,1,376.8,502.1,392.1,546.5,620.0
8,190001005219,2019,1502954,Eldorado do Carajás,15,PA,3,F,18,1.0,1,377.6,443.8,425.0,433.5,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50671,190006119878,2019,2610608,Paudalho,26,PE,0,F,19,1.0,1,419.4,354.0,587.7,461.2,520.0
50672,190006119928,2019,2700300,Arapiraca,27,AL,3,F,26,1.0,1,567.1,513.7,551.0,559.2,460.0
50674,190006120038,2019,2912103,Ibicaraí,29,BA,4,F,17,NaN,2,340.9,497.7,372.7,386.8,260.0
50675,190006120062,2019,2927408,Salvador,29,BA,3,F,18,1.0,2,397.6,438.5,450.2,476.9,300.0


In [143]:
#Tirar alunos ocm nota 0 da selecao
notas_selecao = df_selecao.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0")
notas_selecao

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_ENSINO,TP_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_MT,NU_NOTA_LC,NU_NOTA_REDACAO
4,190001004967,2019,1506807,Santarém,15,PA,3,F,20,1.0,1,537.9,582.2,417.7,585.7,780.0
5,190001005059,2019,1506807,Santarém,15,PA,4,F,17,1.0,2,416.2,569.5,459.9,539.6,660.0
6,190001005138,2019,1508100,Tucuruí,15,PA,3,M,19,NaN,1,526.9,589.7,708.5,588.7,860.0
7,190001005147,2019,1505205,Oeiras do Pará,15,PA,3,F,37,1.0,1,376.8,502.1,392.1,546.5,620.0
8,190001005219,2019,1502954,Eldorado do Carajás,15,PA,3,F,18,1.0,1,377.6,443.8,425.0,433.5,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50671,190006119878,2019,2610608,Paudalho,26,PE,0,F,19,1.0,1,419.4,354.0,587.7,461.2,520.0
50672,190006119928,2019,2700300,Arapiraca,27,AL,3,F,26,1.0,1,567.1,513.7,551.0,559.2,460.0
50674,190006120038,2019,2912103,Ibicaraí,29,BA,4,F,17,NaN,2,340.9,497.7,372.7,386.8,260.0
50675,190006120062,2019,2927408,Salvador,29,BA,3,F,18,1.0,2,397.6,438.5,450.2,476.9,300.0


In [140]:
# salvar a amostra filtrada em disco
notas_selecao.to_csv('data/enem19-sem_Zzero.csv',index=False)

In [136]:
#df_selecao = df_selecao[df_selecao["NU_NOTA_MT"] == 0]
#df_selecao

In [137]:
#df_selecao = df_selecao.drop(notas_zero.index)
#df_selecao

In [144]:
#Numero pessoas que sairam da analise
len(df_selecao)-len(notas_selecao)

27